In [ ]:
%matplotlib inline
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import itertools
import sklearn
from sklearn.cluster import KMeans

In [ ]:
with open("dimredux-challenge.npz", "rb") as f:
    X = np.load(f)
    data = X['data']
    dtraj = X['dtraj']
    
print("got data with shape {}".format(data.shape))

In [ ]:
f = plt.figure()
ax = f.add_subplot(111, projection='3d')
ax.scatter(data[:, 0], data[:, 1], data[:, 2], c=data[:, 2])
plt.show()

In [ ]:
def whiten(x):
    x_meanfree = (x - np.mean(x, axis=0))
    C = (1 / (x.shape[0]-1)) * x_meanfree.T @ x_meanfree

    eigenvalues, W  = np.linalg.eigh(C)
    sig_inv = np.diag(1./np.sqrt(np.abs(eigenvalues) + 1e-6))
    C_sqrt_inv = W @ sig_inv @ W.T

    whitened = x_meanfree @ C_sqrt_inv
    return whitened

In [ ]:
f = plt.figure()
ax = f.add_subplot(111, projection='3d')
ax.scatter(*(whiten(data).T), c=whiten(data)[:,2])
plt.show()

Kullback-Leibler divergence $D_{\text{KL}}(P \| Q) := -\sum_i P(i)\log \frac{Q(i)}{P(i)} = \sum_i P(i)\log \frac{P(i)}{Q(i)}$.

Latent loss $D_{\text{KL}}(P \| \mathcal{N}(0,1) ) = D_\text{KL}(\mathcal{N}(\mu_\text{latent}, \sigma_\text{latent}) \| \mathcal{N}(0,1))$.

In [ ]:
class TVAE(object):
    
    def __init__(self, enc_units, dec_units, ndim=3, latent_dim=2, activation=tf.nn.leaky_relu):
        self.tf_is_traing_pl = tf.placeholder_with_default(True, shape=())
        self.features = tf.placeholder(tf.float32, shape=[None, ndim])
        self.timelagged_features = tf.placeholder(tf.float32, shape=[None, ndim])
        self.latent_dim = latent_dim
        x = tf.layers.dense(self.features, units=enc_units[0], activation=activation)
        x = tf.layers.dropout(x, rate=.5, training=self.tf_is_traing_pl)
        for units in enc_units[1:]:
            x = tf.layers.dense(x, units=units, activation=activation)
            x = tf.layers.dropout(x, rate=.5, training=self.tf_is_traing_pl)
        latent_mean = tf.layers.dense(x, units=latent_dim, activation=None)
        latent_log_std_sq = tf.layers.dense(x, units=latent_dim, activation=None)
        
        samples = tf.random_normal((tf.shape(self.features)[0], latent_dim), mean=0., stddev=1., dtype=tf.float32)
        self.z = latent_mean + tf.sqrt(tf.exp(latent_log_std_sq)) * samples
        
        x = tf.layers.dense(self.z, units=dec_units[0], activation=activation)
        x = tf.layers.dropout(x, rate=.5, training=self.tf_is_traing_pl)
        for units in dec_units[1:]:
            x = tf.layers.dense(x, units=units, activation=activation)
            x = tf.layers.dropout(x, rate=.5, training=self.tf_is_traing_pl)
        self.decoded = tf.layers.dense(x, units=ndim, activation=None)
        
        reconstruction_loss = .5*tf.reduce_sum(tf.pow(self.decoded - self.timelagged_features, 2.0))
        self.reconstruction_loss = tf.reduce_mean(reconstruction_loss)
        latent_loss = -.5 * tf.reduce_sum(1 + latent_log_std_sq - tf.sent_mean = tf.layers.dense(x, units=latent_dim, activation=None)
        latent_logquare(latent_mean) 
                                          - tf.exp(latent_log_std_sq), axis=1)
        self.latent_loss = tf.reduce_mean(latent_loss)
        self.loss = tf.reduce_mean(latent_loss + reconstruction_loss)
        self.train_operation = tf.train.AdamOptimizer(learning_rate=1e-4).minimize(self.loss)
        
    def train(self, session, chunk, chunk_lagged):
        _, loss, reconstruction_loss, latent_loss = session.run(
            (self.train_operation, self.loss, self.reconstruction_loss, self.latent_loss),
            feed_dict={self.features: chunk, self.timelagged_features: chunk_lagged, self.tf_is_traing_pl: True}
        )
        return loss, reconstruction_loss, latent_loss
    
    def predict(self, session, batch):
        return session.run(self.decoded, 
                           feed_dict={self.features: batch, 
                                      self.tf_is_traing_pl: False})
    
    def decode(self, session, z):
        return session.run(self.decoded, 
                           feed_dict={self.z: z, self.tf_is_traing_pl: False})
    
    def encode(self, session, batch):
        return session.run(self.z, feed_dict={self.features: batch, self.tf_is_traing_pl: False})
    
    def generate(self, session, n=1, hidden=None):
        if hidden is None:
            hidden = session.run(tf.random_normal([n, self.latent_dim]))
        return session.run(self.decoded, 
                           feed_dict={self.z: hidden})

In [ ]:
batch_size = 500
n_epochs = 500
lag=1

X = whiten(data[:-lag])
Y = whiten(data[lag:])

all_losses = []
all_reconstruction_losses = []
all_latent_losses = []

tvae = TVAE([300,150], [150,300], ndim=3, latent_dim=1, activation=tf.nn.leaky_relu)
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    saver = tf.train.Saver(max_to_keep=20)

    for epoch in range(n_epochs):
        losses = []
        reconstruction_losses = []
        latent_losses = []
        for ix in range(len(X) // batch_size):
            batch = X[ix*batch_size : min((ix+1)*batch_size, len(X))]
            batch_lagged = Y[ix*batch_size : min((ix+1)*batch_size, len(Y))]
            if len(batch) == len(batch_lagged):
                out = tvae.train(sess, batch, batch_lagged)
                losses.append(out[0])
                reconstruction_losses.append(out[1])
                latent_losses.append(out[2])
        if epoch % 1 == 0:
            print("epoch {}: loss {}".format(epoch, np.mean(losses)))
        if (epoch+1) % 100 == 0:
            save_path = saver.save(sess, "/tmp/tvae/model-{}.ckpt".format(epoch+1))
            print("Model saved in path: {}".format(save_path))
        all_losses.append(np.mean(losses))
        all_reconstruction_losses.append(np.mean(reconstruction_losses))
        all_latent_losses.append(np.mean(latent_losses))

In [ ]:
plt.loglog(all_losses, 'x', label='total')
plt.loglog(all_latent_losses, label='latent')
plt.loglog(all_reconstruction_losses, label='reconstruction')
plt.legend()

In [ ]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    saver.restore(sess, "/tmp/tvae/model-500.ckpt")
    latent = tvae.encode(sess, whiten(data))

In [ ]:
cc = KMeans(n_clusters=4).fit(latent).cluster_centers_

In [ ]:
f, ax = plt.subplots(1,1,figsize=(15, 5))
ax.plot(latent[:1000])
ax.hlines(cc, xmin=0, xmax=1000)

In [ ]:
cc

In [ ]:
guess = np.abs(latent[:, 0, None] - cc.squeeze()[None, :]).argmin(axis=1) #np.array([-1.5, -.5, .5, 1.5])
np.max([np.sum(np.array(p)[dtraj] == guess) for p in itertools.permutations([1,2,3,0])]) * 100. / len(dtraj)

In [ ]:
class TAE(object):
    
    def __init__(self, enc_units, dec_units, ndim=3, latent_dim=2, activation=tf.nn.leaky_relu, dropout=.3):
        self.tf_is_traing_pl = tf.placeholder_with_default(True, shape=())
        self.features = tf.placeholder(tf.float32, shape=[None, ndim])
        self.timelagged_features = tf.placeholder(tf.float32, shape=[None, ndim])
        x = tf.layers.batch_normalization(self.features)
        x = tf.layers.dense(x, units=enc_units[0], activation=activation)
        x = tf.layers.dropout(x, rate=dropout, training=self.tf_is_traing_pl)
        for units in enc_units[1:]:
            x = tf.layers.dense(x, units=units, activation=activation)
            x = tf.layers.dropout(x, rate=dropout, training=self.tf_is_traing_pl)
        
        self.z = tf.layers.dense(x, latent_dim, activation=None)
        
        x = tf.layers.dense(self.z, units=dec_units[0], activation=activation)
        x = tf.layers.dropout(x, rate=dropout, training=self.tf_is_traing_pl)
        for units in dec_units[1:]:
            x = tf.layers.dense(x, units=units, activation=activation)
            x = tf.layers.dropout(x, rate=dropout, training=self.tf_is_traing_pl)
        self.decoded = tf.layers.dense(x, units=ndim, activation=None)
        
        self.loss = tf.losses.mean_squared_error(self.timelagged_features, self.decoded)
        self.train_operation = tf.train.AdamOptimizer(learning_rate=5e-5).minimize(self.loss)
        
    def train(self, session, X, Y):
        self.training = True
        _, loss, = session.run(
            (self.train_operation, self.loss),
            feed_dict={self.features: X, self.timelagged_features: Y, self.tf_is_traing_pl: True}
        )
        self.training = False
        return loss
    
    def predict(self, session, batch):
        return session.run(self.decoded, feed_dict={self.features: batch, self.tf_is_traing_pl: False})
    
    def decode(self, session, z):
        return session.run(self.decoded, feed_dict={self.z: z, self.tf_is_traing_pl: False})
    
    def encode(self, session, batch):
        return session.run(self.z, feed_dict={self.features: batch, self.tf_is_traing_pl: False})

In [ ]:
batch_size = 500
n_epochs = 2000
lag=1

all_losses = []
all_reconstruction_losses = []
all_latent_losses = []

X = whiten(data[:-lag])
Y = whiten(data[lag:])

tae = TAE([200,100], [100,200], ndim=3, latent_dim=1, activation=tf.nn.leaky_relu, dropout=.5)
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    saver = tf.train.Saver(max_to_keep=20)

    for epoch in range(n_epochs):
        losses = []
        for ix in range(len(data) // batch_size):
            batch = X[ix*batch_size : min((ix+1)*batch_size, len(X))]
            batch_lagged = Y[ix*batch_size : min((ix+1)*batch_size, len(Y))]
            if len(batch) == len(batch_lagged):
                out = tae.train(sess, batch, batch_lagged)
                losses.append(out)
            
        if epoch % 1 == 0:
            print("epoch {}: loss {}".format(epoch, np.mean(losses)))
        if (epoch+1) % 100 == 0:
            save_path = saver.save(sess, "/tmp/tae/model-{}.ckpt".format(epoch+1))
            print("Model saved in path: {}".format(save_path))
        all_losses.append(np.mean(losses))

In [ ]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    saver.restore(sess, "/tmp/tae/model-2000.ckpt")
    latent = tae.encode(sess, whiten(data))  # whiten(data)

In [ ]:
cc = KMeans(n_clusters=4).fit(latent).cluster_centers_
# cc = [-0.05, .07, .2, -.2]
f, ax = plt.subplots(1,1, figsize=(15, 5))
ax.plot(latent[:1000])
ax.hlines(cc, xmin=0, xmax=1000)
plt.show()

In [ ]:
guess = np.abs(latent[:, 0, None] - np.array(cc).squeeze()[None, :]).argmin(axis=1)
perms = [np.sum(np.array(p)[dtraj] == guess) for p in itertools.permutations([1,2,3,0])]
percent = np.max(perms) * 100. / len(dtraj)
print("{:.4f}%".format(percent))